In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

triplet loss function https://pytorch.org/docs/stable/nn.html#tripletmarginloss

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
import pandas as pd

In [4]:
import  datatset
from torch.utils.data import DataLoader

In [5]:
# DataLoader

In [11]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [12]:
label_counts  = df["category_id"].value_counts()

In [13]:
num_calsses = len(label_counts) + 1

In [14]:
labels_ordered = label_counts.keys().tolist()

In [15]:
num_calsses

32094

In [16]:
ds = datatset.HerbTripletDataSet(df_train, TRAIN_PATH,128,label="category_id")

In [18]:
data_loader = DataLoader(ds, batch_size=16, shuffle=True)

In [21]:
# Tracking

In [19]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/"logs/run-2")

In [20]:
import metrics
from train import Training

In [21]:
acc =  metrics.Accuracy()
manager = metrics.MetricManager([acc], writer)

In [22]:
# Train

In [39]:
class Trainer(Training):
    def __init(self,  metrics, loss, optim, data, epochs, model, save_dir):
        super(self).__init__(self,  metrics, loss, optim, data, epochs, model, save_dir)
        
    def train_step(self, batch):
        positve, negative, anchor = batch
        logits = self.model(positve), self.model(negative), self.model(anchor) # gotta be a better way
        loss = self.loss(*logits)
        print(f'Loss: {loss.item()}')
        
#         if not self.metrics.writer:
#             self.metrics.writer.add_scalar("loss", loss.item(), self.step)

        self.optim.zero_grad()  # zero gradients
        loss.backward()  # calculate gradients
        self.optim.step()  # updated weights


In [40]:
import torch
import torch.nn as nn
import torchvision.models as models

In [41]:
def get_model():
    base = models.resnet18(pretrained=True)
    base.fc  = nn.Linear(in_features=512, out_features=512)
    return base.cuda()

In [42]:
model = get_model()

In [43]:
criterion = nn.TripletMarginLoss()

In [44]:
optimizer = torch.optim.Adam(model.parameters())

In [45]:
model_path=TRAIN_PATH/"models/"

In [46]:
trainer = Trainer(manager, criterion, optimizer, data_loader, 3, model, model_path )

In [47]:
trainer.train_loop()

Epoch 0/3


Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0
Loss: 1.0


KeyboardInterrupt: 